# Install


In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from dotenv import load_dotenv
import pandas as pd

import process_transactions
import requisition
import utils

load_dotenv("../user.env")

True

# Get user data

In [7]:
user_data = await requisition.get_user_data()

2024-02-01 20:20:01.921 | INFO     | requisition:_get_requisition_id:29 - found 1 requisition_ids
2024-02-01 20:20:01.922 | INFO     | services.nordigen.account_api:get_latest_data:226 - today's data exists
/var/folders/rp/f2ppsq3x6s33kb4f3qph_x1h0000gn/T/ipykernel_22008/2196920193.py:1: RuntimeWarning: coroutine 'get_user_data' was never awaited
  user_data = await requisition.get_user_data()


# Get updated transactions

In [9]:
await process_transactions.main()
user_data.transactions = utils.read_json(process_transactions.SAVE_PATH)

2024-02-01 20:21:13.601 | INFO     | requisition:_get_requisition_id:29 - found 1 requisition_ids
2024-02-01 20:21:13.602 | INFO     | services.nordigen.account_api:get_latest_data:226 - today's data exists


2024-02-01 20:21:13.682 | DEBUG    | process_transactions:combine_transactions:117 - joining historic and live transaction tables
2024-02-01 20:21:13.845 | INFO     | process_transactions:_enrich_transactions_dataframe:33 - Enriching dataframe
2024-02-01 20:21:13.850 | WARNING  | enrich.counterparty:get_counterparty:244 - unknown proprietaryBankTransactionCode: FASTER PAYMENT DEBIT REVERSAL 
2024-02-01 20:21:13.857 | INFO     | enrich.categories:categorize_transactions:35 - Adding transactions category


In [49]:
account_id = user_data.requisition.accounts[0]

In [53]:
user_data.account_balances.balances.get(account_id)

AccountBalances(account_name='joint account', account_id='590300bd-3daf-4d5e-9274-7a3782261f7e', balances={'interimAvailable': Balance(balance_amount=BalanceAmount(amount='17054.26', currency='GBP'), balance_type='interimAvailable', reference_date='2024-02-01'), 'interimBooked': Balance(balance_amount=BalanceAmount(amount='17054.26', currency='GBP'), balance_type='interimBooked', reference_date='2024-02-01')})

In [67]:
df = pd.concat([pd.json_normalize(user_data.account_balances.get_balances_by_account_id(account_id).model_dump().get("balances")) for account_id in  user_data.requisition.accounts])
df

,interimAvailable.balance_amount.amount,interimAvailable.balance_amount.currency,interimAvailable.balance_type,interimAvailable.reference_date,interimBooked.balance_amount.amount,interimBooked.balance_amount.currency,interimBooked.balance_type,interimBooked.reference_date,forwardAvailable.balance_amount.amount,forwardAvailable.balance_amount.currency,forwardAvailable.balance_type,forwardAvailable.reference_date,openingCleared.balance_amount.amount,openingCleared.balance_amount.currency,openingCleared.balance_type,openingCleared.reference_date,previouslyClosedBooked.balance_amount.amount,previouslyClosedBooked.balance_amount.currency,previouslyClosedBooked.balance_type,previouslyClosedBooked.reference_date
0,17054.26,GBP,interimAvailable,2024-02-01,17054.26,GBP,interimBooked,2024-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4532.27,GBP,forwardAvailable,2024-02-01,-733.71,GBP,openingCleared,2024-02-01,-92.06,GBP,previouslyClosedBooked,2024-01-23
0,23.14,GBP,interimAvailable,2024-02-01,23.14,GBP,interimBooked,2024-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
df

,interimAvailable.balance_amount.amount,interimAvailable.balance_amount.currency,interimAvailable.balance_type,interimAvailable.reference_date,interimBooked.balance_amount.amount,interimBooked.balance_amount.currency,interimBooked.balance_type,interimBooked.reference_date
0,17054.26,GBP,interimAvailable,2024-02-01,17054.26,GBP,interimBooked,2024-02-01


# Query


In [ ]:
import pandas as pd

In [ ]:
records = user_data.query_transactions_by_transaction_code(
    "FASTER PAYMENT DEBIT REVERSAL"
)